In [ ]:
# save this to a file - it will be handy for MR testing later
with open('test.txt','w') as outfile:
    outfile.write('hot chocolate cocoa beans\n')
    outfile.write('cocoa ghana africa\n')
    outfile.write('beans harvest ghana\n')
    outfile.write('cocoa butter\n')
    outfile.write('butter truffles\n')
    outfile.write('sweet chocolate\n')
    outfile.write('sweet sugar\n')
    outfile.write('sugar cane brazil\n')
    outfile.write('sweet sugar beet\n')
    outfile.write('sweet cake icing\n')
    outfile.write('cake black forest\n')
# if you take out rounding you can see that these values are not really exact
np.around(r, 3)



In [ ]:
import re
from math import log
import numpy as np

# read the documents. Each document consists of a list of words.
documents = []
with open('test.txt', 'r') as docfile:
    for line in docfile.readlines():
        documents.append(re.split(' ', line.strip()))

classes = 2
r = [[None] * len(documents), [None] * len(documents)]

# set our initial conditions (r_6,1 = 1.0 and r_7,1 = 0.0 and the converse for the other class)
r[0][5] = r[1][6] = 1.0
r[0][6] = r[1][5] = 0.0

# initialize the priors
alpha = [0.0] * classes

epsilon = 0.0001

# conditional term probabilities
qm = {}

# compute alphas - Equation 3
def compute_alphas(alphas):
    for k in range(classes):
        alphas[k] = sum([x for x in r[k] if x is not None])/len([x for x in r[k] if x is not None])


# compute inverted postings list
# this is handy for the computation of the qm's
def compute_postings():
    postings = {}
    for i in range(len(documents)):
        if r[0][i] is not None:
            for word in documents[i]:
                if word not in postings:
                    postings[word] = [i]
                else:
                    if i not in postings[word]:
                        postings[word].append(i)
    return postings

# compute qm's - Equation 2
def compute_next_qms(qm):
    for k in range(classes):
        for i in range(len(r[k])):
            if r[k][i] is not None:
                for word in documents[i]:
                    if word not in qm:
                        qm[word] = {k: sum([r[k][j] for j in postings[word]]) / \
                                    sum([x for x in r[k] if x is not None])}
                    else:
                        qm[word][k] = sum([r[k][j] for j in postings[word]]) / \
                                    sum([x for x in r[k] if x is not None])
    

# compute next iteration of r's - Equation 1a
# note - need to do log(probability)
def compute_next_r(rs):
    for i in range(len(documents)):
        vocab_words_in_doc = []
        p = np.zeros((classes, 2))
        
        # find all vocab words in the doc. Note: there may be none.
        for word in documents[i]:
            if word in qm:
                vocab_words_in_doc.append(word)
                for k in range(classes):
                    p[k][0] += np.log(qm[word][k] + epsilon)
        if len(vocab_words_in_doc) > 0:
            # find all vocab words not in doc for all classes at the same time
            for word in qm:
                if word not in vocab_words_in_doc:
                    for k in range(classes):
                        p[k][1] += np.log(1-qm[word][k] + epsilon)
                        
            # compute the denominator of Equation 1a for all classes
            denom = 0.0
            for k in range(classes):
                denom += alpha[k]*np.exp(p[k][0]+p[k][1])
                
            # compute the new r of Equation 1a for all classes
            for k in range(classes):
                rs[k][i] = alpha[k]*np.exp(p[k][0]+p[k][1])/denom

        else:
            # set to prior in case of no information
            for k in range(classes):
                rs[k][i] = alpha[k]

# iterate
for _ in range(25):
    compute_alphas(alpha)
    postings = compute_postings()
    compute_next_qms(qm)
    compute_next_r(r)
# if you take out rounding you can see that these values are not really exact
np.around(r, 3)



In [ ]:
# if you take out rounding you can see that these values are not really exact
np.around(r, 3)

